In [1]:
import pandas as pd
import numpy as np

In [2]:
ROOT = '../../data/raw/house/'

In [3]:
df = pd.read_csv(ROOT + 'train.csv', index_col='Id')

In [4]:
# Фічі які не object/str
numerical = [x for x in df.columns if df[x].dtype != 'O']

In [77]:
# X - наші фічі
# y - наш таргет
X = df[numerical].drop('SalePrice', axis=1).fillna(0) # відфільтровуємо object фічі, убираємо таргет з X
y = np.log(df['SalePrice']) # логарифм для скейлінгу великого таргету

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [76]:
# Ініціалізація МЛ моделі
model = LinearRegression()

### 1. Holdout

In [67]:
# Робимо holdout
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [71]:
# Тренуємо модель на навчальній виборці
model.fit(X_train, y_train)

LinearRegression()

In [72]:
# Ранимо модель на тест сеті, отримаємо предікшени моделі
preds = model.predict(X_test)

In [74]:
# Рахуємо точність моделі на тесті
mean_squared_error(y_test, preds)

np.float64(1062554651.2497369)

### 2. KFold

In [78]:
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm # tqdm для progress barу

In [79]:
# Ініціалізація KFold класу з заданими параметрами
k = 5
kf = KFold(n_splits=k, random_state=42, shuffle=True)

In [80]:
# Ініціалізація МЛ моделі
model = LinearRegression()

In [83]:
# X - наші фічі
# y - наш таргет
X = df[numerical].drop('SalePrice', axis=1).fillna(0) # відфільтровуємо object фічі, убираємо таргет з X
y = np.log(df['SalePrice']) # логарифм для скейлінгу великого таргету

In [84]:
# pandas -> numpy
# так буде простіше селектити далі по індексам
X = X.values
y = y.values

In [85]:
mse_score = [] # тут сторимо результати точності по всім фолдам
 
for train_index , test_index in tqdm(kf.split(X)): # робимо спліт через for loop. отримуємо індекси трейна та теста на поточному шагу
    X_train , X_test = X[train_index,:], X[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train) # тренування
    pred_values = model.predict(X_test) # інференс на тесті
     
    mse = mean_squared_error(pred_values , y_test) # оцінка точності на поточному тесті
    mse_score.append(mse)

mse_score = np.array(mse_score) # list -> np.array

0it [00:00, ?it/s]

In [86]:
mse_score

array([0.02275559, 0.01790913, 0.05720151, 0.0193098 , 0.01759958])